In [153]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
import scipy.sparse as sp
from itertools import islice, cycle
from more_itertools import pairwise
from tqdm.auto import tqdm

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))
os.chdir("C:\\Users\\Администратор.WIN-U1NLG8MM702\\Documents\\Katerina\\Datasets for analysis")
# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [154]:

interactions = pd.read_csv('../input/mts-ml-summer-school/interactions.csv')
items = pd.read_csv('../input/mts-ml-summer-school/items.csv')
users = pd.read_csv('../input/mts-ml-summer-school/users.csv')

In [155]:
interactions.shape, items.shape, users.shape

((1562617, 5), (63758, 5), (137254, 3))

### Предобработка

In [156]:
interactions['start_date'] = pd.to_datetime(interactions['start_date'])

In [157]:
duplicates = interactions.duplicated(subset=['user_id', 'item_id'], keep=False)
df_duplicates = interactions[duplicates].sort_values(by=['user_id', 'start_date'])
interactions = interactions[~duplicates]

In [158]:
df_duplicates = df_duplicates.groupby(['user_id', 'item_id']).agg({
    'progress': 'max',
    'rating': 'max',
    'start_date': 'min'
})
interactions = interactions.append(df_duplicates.reset_index(), ignore_index=True)

In [159]:
interactions['progress'] = interactions['progress'].astype(np.int8)
interactions['rating'] = interactions['rating'].astype(pd.SparseDtype(np.float32, np.nan))

In [160]:
interactions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1562505 entries, 0 to 1562504
Data columns (total 5 columns):
 #   Column      Non-Null Count    Dtype               
---  ------      --------------    -----               
 0   user_id     1562505 non-null  int64               
 1   item_id     1562505 non-null  int64               
 2   progress    1562505 non-null  int8                
 3   rating      323563 non-null   Sparse[float32, nan]
 4   start_date  1562505 non-null  datetime64[ns]      
dtypes: Sparse[float32, nan](1), datetime64[ns](1), int64(2), int8(1)
memory usage: 39.7 MB


In [161]:
interactions.to_pickle('interactions_preprocessed.pickle')

In [162]:
users.head()

,user_id,age,sex
0,0,45_54,1.0
1,1,25_34,0.0
2,2,45_54,0.0
3,3,65_inf,0.0
4,4,18_24,0.0


In [163]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype  
---  ------   --------------   -----  
 0   user_id  137254 non-null  int64  
 1   age      137244 non-null  object 
 2   sex      135640 non-null  float64
dtypes: float64(1), int64(1), object(1)
memory usage: 3.1+ MB


In [164]:
users.nunique()

user_id    137254
age             6
sex             2
dtype: int64

In [165]:
users['age'] = users['age'].astype('category')
users['sex'] = users['sex'].astype(pd.SparseDtype(np.float32, np.nan))


In [166]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137254 entries, 0 to 137253
Data columns (total 3 columns):
 #   Column   Non-Null Count   Dtype               
---  ------   --------------   -----               
 0   user_id  137254 non-null  int64               
 1   age      137244 non-null  category            
 2   sex      135640 non-null  Sparse[float32, nan]
dtypes: Sparse[float32, nan](1), category(1), int64(1)
memory usage: 2.2 MB


In [167]:
interaction_users = interactions['user_id'].unique()

common_users = len(np.intersect1d(interaction_users, users['user_id']))
users_only_in_interaction = len(np.setdiff1d(interaction_users, users['user_id']))
users_only_features = len(np.setdiff1d(users['user_id'], interaction_users))
total_users = common_users + users_only_in_interaction + users_only_features
print(f'Кол-во пользователей - {total_users}')
print(f'Кол-во пользователей c взаимодействиями и фичами - {common_users} ({common_users / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c взаимодействиями - {users_only_in_interaction} ({users_only_in_interaction / total_users * 100:.2f}%)')
print(f'Кол-во пользователей только c фичами - {users_only_features} ({users_only_features / total_users * 100:.2f}%)')

Кол-во пользователей - 164771
Кол-во пользователей c взаимодействиями и фичами - 130808 (79.39%)
Кол-во пользователей только c взаимодействиями - 27517 (16.70%)
Кол-во пользователей только c фичами - 6446 (3.91%)


In [168]:
users.to_pickle('users_preprocessed.pickle')

In [169]:
items.head()

,id,title,genres,authors,year
0,248031,Ворон-челобитчик,"Зарубежные детские книги,Сказки,Зарубежная кла...",Михаил Салтыков-Щедрин,1886
1,256084,Скрипка Ротшильда,"Классическая проза,Литература 19 века,Русская ...",Антон Чехов,1894
2,134166,Испорченные дети,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1869
3,281311,Странный человек,"Пьесы и драматургия,Литература 19 века",Михаил Лермонтов,1831
4,213473,Господа ташкентцы,"Зарубежная классика,Классическая проза,Литерат...",Михаил Салтыков-Щедрин,1873


In [170]:
items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   id       63758 non-null  int64 
 1   title    63758 non-null  object
 2   genres   63753 non-null  object
 3   authors  56700 non-null  object
 4   year     49508 non-null  object
dtypes: int64(1), object(4)
memory usage: 2.4+ MB


In [171]:
items.nunique()
#Почему колонка year типа object, а не int?

id         63758
title      58093
genres     11091
authors    17024
year        1074
dtype: int64

In [172]:
for col in ['genres', 'authors', 'year']:
    items[col] = items[col].astype('category')

In [173]:
items.info(memory_usage='full')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 63758 entries, 0 to 63757
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype   
---  ------   --------------  -----   
 0   id       63758 non-null  int64   
 1   title    63758 non-null  object  
 2   genres   63753 non-null  category
 3   authors  56700 non-null  category
 4   year     49508 non-null  category
dtypes: category(3), int64(1), object(1)
memory usage: 2.5+ MB


In [174]:
interaction_items = interactions['item_id'].unique()

common_items = len(np.intersect1d(interaction_items, items['id']))
items_only_in_interaction = len(np.setdiff1d(interaction_items, items['id']))
items_only_features = len(np.setdiff1d(items['id'], interaction_items))
total_items = common_items + items_only_in_interaction + items_only_features
print(f'Кол-во книг - {total_items}')
print(f'Кол-во книг c взаимодействиями и фичами - {common_items} ({common_items / total_items * 100:.2f}%)')
print(f'Кол-во книг только c взаимодействиями - {items_only_in_interaction} ({items_only_in_interaction / total_items * 100:.2f}%)')
print(f'Кол-во книг только c фичами - {items_only_features} ({items_only_features / total_items * 100:.2f}%)')

Кол-во книг - 63758
Кол-во книг c взаимодействиями и фичами - 63758 (100.00%)
Кол-во книг только c взаимодействиями - 0 (0.00%)
Кол-во книг только c фичами - 0 (0.00%)


In [175]:
items.to_pickle('items_preprocessed.pickle')

In [176]:
users_inv_mapping = dict(enumerate(interactions['user_id'].unique()))
users_mapping = {v: k for k, v in users_inv_mapping.items()}
len(users_mapping)

158325

In [177]:
items_inv_mapping = dict(enumerate(interactions['item_id'].unique()))
items_mapping = {v: k for k, v in items_inv_mapping.items()}
len(items_mapping)

63758

In [178]:
items['title'] = items['title'].str.lower()

In [179]:
item_titles = pd.Series(items['title'].values, index=items['id']).to_dict()

In [180]:
title_items = items.groupby('title')['id'].agg(list)

In [181]:
title_count = title_items.map(len)
title_count.value_counts()

1     53608
2      3623
3       488
4       151
5        61
6        23
7        17
8         8
9         5
18        1
10        1
11        1
12        1
35        1
13        1
15        1
51        1
Name: id, dtype: int64

In [182]:
title_items[title_count > 1].tail()

title
яр                           [108212, 252893]
ярмарка тщеславия    [323949, 351537, 324231]
ярослав мудрый                [358342, 83942]
ярость тьмы                   [324469, 74554]
ящик пандоры                  [93562, 164956]
Name: id, dtype: object

In [183]:
items[items['title'] == 'ящик пандоры']

,id,title,genres,authors,year
39115,93562,ящик пандоры,"Любовно-фантастические романы,Научная фантастика",Филипп Хорват,2017
54893,164956,ящик пандоры,"Мистика,Современная зарубежная литература",Бернар Вербер,2018


In [184]:
items[items['title'] == '451 градус по фаренгейту']

,id,title,genres,authors,year
49284,99550,451 градус по фаренгейту,"Социальная фантастика,Зарубежная фантастика,На...",Рэй Брэдбери,"1951, 1953, 1967"


In [185]:
interactions['rating'] = np.array(interactions['rating'].values, dtype=np.float32)

interactions.loc[interactions['item_id'].isin([99550])].groupby('item_id').agg({
    'progress': np.size,
    'rating': ['mean'],
    'start_date': ['min', 'max'],
})

progress rating start_date           
            size   mean        min        max
item_id                                      
99550         55    3.4 2018-01-26 2019-12-30

### Validation

In [186]:
def compute_metrics(df_true, df_pred, top_N):
    result = {}
    test_recs = df_true.set_index(['user_id', 'item_id']).join(df_pred.set_index(['user_id', 'item_id']))
    test_recs = test_recs.sort_values(by=['user_id', 'rank'])

    test_recs['users_item_count'] = test_recs.groupby(level='user_id')['rank'].transform(np.size)
    test_recs['reciprocal_rank'] = (1 / test_recs['rank']).fillna(0)
    test_recs['cumulative_rank'] = test_recs.groupby(level='user_id').cumcount() + 1
    test_recs['cumulative_rank'] = test_recs['cumulative_rank'] / test_recs['rank']
    
    users_count = test_recs.index.get_level_values('user_id').nunique()
    for k in range(1, top_N + 1):
        hit_k = f'hit@{k}'
        test_recs[hit_k] = test_recs['rank'] <= k
        result[f'Precision@{k}'] = (test_recs[hit_k] / k).sum() / users_count
        result[f'Recall@{k}'] = (test_recs[hit_k] / test_recs['users_item_count']).sum() / users_count

    result[f'MAP@{top_N}'] = (test_recs["cumulative_rank"] / test_recs["users_item_count"]).sum() / users_count
    result[f'MRR'] = test_recs.groupby(level='user_id')['reciprocal_rank'].max().mean()
    return pd.Series(result)

In [187]:
train=interactions

### Baseline

In [190]:
class PopularRecommender():
    def __init__(self, max_K=100, days=30, item_column='item_id', dt_column='date'):
        self.max_K = max_K
        self.days = days
        self.item_column = item_column
        self.dt_column = dt_column
        self.recommendations = []
        
    def fit(self, df):
        min_date = df[self.dt_column].max().normalize() - pd.DateOffset(days=self.days)
        self.recommendations = df.loc[df[self.dt_column] > min_date, self.item_column].value_counts().head(self.max_K).index.values
    
    def recommend(self, users=None, N=10):
        recs = self.recommendations[:N]
        if users is None:
            return recs
        else:
            return list(islice(cycle([recs]), len(users)))

### Варьируем дни

In [263]:
pop_model = PopularRecommender(days=70, dt_column='start_date')
pop_model.fit(interactions)

In [264]:
top10_recs = pop_model.recommend()
top10_recs

array([178529, 276903, 283713, 184549, 267817, 187325, 168963, 143175,
        79499, 352049], dtype=int64)

In [265]:
sample = pd.read_csv('../input/mts-ml-summer-school/sample_submission.csv')

In [266]:
sample.head()

,Id,Predicted
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...


In [267]:
recs = pd.DataFrame({'user_id': sample['Id'].unique()})
top_N = 10
recs['item_id'] = pop_model.recommend(recs['user_id'], N=top_N)
recs.head()

,user_id,item_id
0,10001,"[178529, 276903, 283713, 184549, 267817, 18732..."
1,10002,"[178529, 276903, 283713, 184549, 267817, 18732..."
2,100152,"[178529, 276903, 283713, 184549, 267817, 18732..."
3,100197,"[178529, 276903, 283713, 184549, 267817, 18732..."
4,100284,"[178529, 276903, 283713, 184549, 267817, 18732..."


In [268]:
recs = recs.explode('item_id')
recs.head(top_N + 2)

,user_id,item_id
0,10001,178529
0,10001,276903
0,10001,283713
0,10001,184549
0,10001,267817
0,10001,187325
0,10001,168963
0,10001,143175
0,10001,79499
0,10001,352049


In [269]:
recs['rank'] = recs.groupby('user_id').cumcount() + 1
recs.head(top_N + 2)

,user_id,item_id,rank
0,10001,178529,1
0,10001,276903,2
0,10001,283713,3
0,10001,184549,4
0,10001,267817,5
0,10001,187325,6
0,10001,168963,7
0,10001,143175,8
0,10001,79499,9
0,10001,352049,10


### Функция для конвертации датафрейма с рекомендациями в нужный формат для сабмита

In [270]:
def make_submission(df, id_col='user_id', predicted_col='item_id'):
    
    df_copy = df.copy()
    df_copy.loc[:, id_col] = df_copy[id_col].astype(str)
    df_copy.loc[:, predicted_col] = df_copy[predicted_col].astype(str)
    df_copy = df_copy.groupby(id_col, as_index=False).agg({predicted_col: lambda x: ' '.join(list(x))})
    df_copy.rename(columns={id_col: 'Id', predicted_col: 'Predicted'}, inplace=True)
    
    return df_copy[['Id', 'Predicted']]

In [271]:
recs.head()

,user_id,item_id,rank
0,10001,178529,1
0,10001,276903,2
0,10001,283713,3
0,10001,184549,4
0,10001,267817,5


In [272]:
sample_recs = make_submission(recs)
sample_recs.head()

,Id,Predicted
0,10001,178529 276903 283713 184549 267817 187325 1689...
1,10002,178529 276903 283713 184549 267817 187325 1689...
2,100152,178529 276903 283713 184549 267817 187325 1689...
3,100197,178529 276903 283713 184549 267817 187325 1689...
4,100284,178529 276903 283713 184549 267817 187325 1689...


In [274]:
sample_recs.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 3074 entries, 0 to 3073
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   Id         3074 non-null   object
 1   Predicted  3074 non-null   object
dtypes: object(2)
memory usage: 72.0+ KB


In [275]:
sample.explode('Predicted')

,Id,Predicted
0,10001,313 253 245 205 187 168 155 149 146 128 128 12...
1,10002,313 253 245 205 187 168 155 149 146 128 128 12...
2,100152,313 253 245 205 187 168 155 149 146 128 128 12...
3,100197,313 253 245 205 187 168 155 149 146 128 128 12...
4,100284,313 253 245 205 187 168 155 149 146 128 128 12...
...,...,...
3069,99734,313 253 245 205 187 168 155 149 146 128 128 12...
3070,99757,313 253 245 205 187 168 155 149 146 128 128 12...
3071,99772,313 253 245 205 187 168 155 149 146 128 128 12...
3072,99827,313 253 245 205 187 168 155 149 146 128 128 12...


In [276]:
submission_pop.to_csv('submission_pop.csv', index=False)